In [45]:
print('Hello')

Hello


In [85]:
from datetime import datetime, timedelta
import time
import pandas as pd

In [86]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path='../credential/Xpring Dev Sandbox-edb2a98acf17.json'
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [47]:
key_path

'../credential/Xpring Dev Sandbox-edb2a98acf17.json'

In [48]:
import os.path
from os import path

path.exists(key_path)

True

In [49]:
def gbq_query(query,cred, query_params=None):
    """
    Run a query against Google Big Query, returning a pandas dataframe of the result.

    Parameters
    ----------
    query: str
        The query string
    cred: obj
        Credential class instance
    
    query_params: list, optional
        The query parameters to pass into the query string
    """
    client = bigquery.Client(credentials=cred)
    job_config = bigquery.QueryJobConfig()
    job_config.query_parameters = query_params
    return client.query(query, job_config=job_config).to_dataframe()

In [50]:
start_date='2019-12-15'
end_date='2019-12-26'

In [51]:
query = """
Select 
  TIMESTAMP(l.CloseTime) as `TimeStamp`,
  t.Account,t.Destination,t.Fee,t.TxnSignature,t.AmountXRP,t.LedgerIndex
FROM
  `xrpledgerdata.fullhistory.transactions`t
JOIN
  `xrpledgerdata.fullhistory.ledgers` l
  on t.LedgerIndex=l.LedgerIndex
where t.TransactionResult = "tesSUCCESS"
AND TxnSignature IS NOT NULL
AND Destination IS NOT NULL
AND AmountXRP  IS NOT NULL
AND TIMESTAMP(l.CloseTime) >=TIMESTAMP(@start_date)
AND TIMESTAMP(l.CloseTime) <=TIMESTAMP(@end_date)


ORDER BY TimeStamp
LIMIT 200000;
"""
query_params = [
    bigquery.ScalarQueryParameter("start_date", "STRING", start_date),
    bigquery.ScalarQueryParameter("end_date", "STRING", end_date)
]


In [96]:
xrp = gbq_query(query,credentials,query_params)
xrp

,TimeStamp,Account,Destination,Fee,TxnSignature,AmountXRP,LedgerIndex
0,2019-12-15 00:00:00+00:00,rPsmHDMkheWZvbAkTA8A9bVnUdadPn7XBK,rUMhAy8zECaY9u6ntqTAXFu1t8gixCeHka,10,304502210089A1985CB7AF1F667C61405FB4A609F06DEB...,48560300000,52058308
1,2019-12-15 00:00:01+00:00,rw2ciyaNshpHe7bCHo4bRWq6pqqynnWKQg,rLNaPoKeeBjZe2qs6x52yVPZpZ8td4dc6w,40,3045022100AEF609203571841221F6D13CD5991EE66AAF...,50000083642,52058309
2,2019-12-15 00:00:01+00:00,rEvoRfi83Hc5gxqpwRtc9jh7Z4C2hECCWY,rnuPTVikw8HKK4hBGCtnq2J2433VYaZPZQ,12,304402203E8C80A7E4F36EEB754BCA9EF3CB2D5B2BFA08...,999999,52058309
3,2019-12-15 00:00:10+00:00,ryBANkk28Mj71jRKAkt13U1X9ubztsGWZ,rwU8rAiE2eyEPz3sikfbHuqCuiAtdXqa2v,7707,304402203D711CFEB876FFC22375F0C8C3CC3B1DB8952D...,15728776000,52058310
4,2019-12-15 00:00:11+00:00,rLNaPoKeeBjZe2qs6x52yVPZpZ8td4dc6w,rwpMvfxoodXggJ1g4qv6MWAPQqWDwQyHUW,40,3045022100F4BEBE96EF6610512BD07D789C1E2629342F...,1162250116,52058311
...,...,...,...,...,...,...,...
199995,2019-12-24 00:34:32+00:00,rJb5KsHsDHF1YS5B5DU6QCkH5NsPaKQTcy,rU2mEJSLqBRkYLVTv55rFTgQajkLTnT6mA,200000,30440220181D2071B07AD340FE60D46FDA0A08F14916FE...,850499000,52256435
199996,2019-12-24 00:34:32+00:00,rJb5KsHsDHF1YS5B5DU6QCkH5NsPaKQTcy,r35XtHaHFHcQ3uhAB8JLNRFWcCPCKecABp,200000,3045022100D5B78A66FE2052731473C28A64EC4D3735EB...,1921750000,52256435
199997,2019-12-24 00:34:32+00:00,rJb5KsHsDHF1YS5B5DU6QCkH5NsPaKQTcy,rMvYS27SYs5dXdFsUgpvv1CSrPsCz7ePF5,200000,3045022100D1F94C4DC2104F39D8F58709545349E26C73...,46423750000,52256435
199998,2019-12-24 00:34:40+00:00,rMvYS27SYs5dXdFsUgpvv1CSrPsCz7ePF5,rEeEWeP88cpKUddKk37B2EZeiHBGiBXY3,1000,3045022100ED4B2301191FCA69E37623F45A13233F22A2...,134782688834,52256436


In [97]:
def datetotime(col):
    #returns a list of datetime objects
    #takes in a df column
    x=[]
    for entry in col:
        x.append(entry.to_pydatetime())
    return x

In [98]:
xrp['TimeStamp']=datetotime(xrp['TimeStamp'])

In [106]:
dir(xrp['TimeStamp'][1])

['__add__',
 '__array_priority__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rsub__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '_date_attributes',
 '_date_repr',
 '_get_date_name_field',
 '_get_start_end_field',
 '_has_time_component',
 '_repr_base',
 '_round',
 '_short_repr',
 '_time_repr',
 'asm8',
 'astimezone',
 'ceil',
 'combine',
 'ctime',
 'date',
 'day',
 'day_name',
 'dayofweek',
 'dayofyear',
 'days_in_month',
 'daysinmonth',
 'dst',
 'floor',
 'freq',
 'freqstr',
 'fromordinal',
 'fromtimestamp',
 'hour',
 'is_leap_year',
 'is_month_end',
 'is_month_start',
 'is_quarter_end',
 'is_quarter_start',
 'is_year_end',
 'is_year_start',
 'isocalendar',
 'is

In [105]:
xrp['TimeStamp']

0        2019-12-15 00:00:00+00:00
1        2019-12-15 00:00:01+00:00
2        2019-12-15 00:00:01+00:00
3        2019-12-15 00:00:10+00:00
4        2019-12-15 00:00:11+00:00
                    ...           
199995   2019-12-24 00:34:32+00:00
199996   2019-12-24 00:34:32+00:00
199997   2019-12-24 00:34:32+00:00
199998   2019-12-24 00:34:40+00:00
199999   2019-12-24 00:34:40+00:00
Name: TimeStamp, Length: 200000, dtype: datetime64[ns, UTC]

In [103]:
datetime1=xrp['TimeStamp'][1]

In [107]:
datetime1.to_pydatetime()

datetime.datetime(2019, 12, 15, 0, 0, 1, tzinfo=<UTC>)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
